In [1]:
# Import the required packeages
import pandas as pd
import urllib3
urllib3.disable_warnings()
from pandas.io.json import json_normalize
from typing import List, Dict
import sys
import os
import numpy as np
import concurrent.futures
import json
import itertools
from aiohttp import ClientSession
import asyncio
import dask.dataframe as dd

In [2]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/meena/dev/pyPrediktorMapClient/src/pyprediktormapclient


In [3]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

In [4]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opc_url = "http://10.241.68.86:13371/"

# Model index API
mdx = ModelIndex(url=model_index_url)

# OPC UA API
opc = OPC_UA(url=opc_url)

In [5]:
# Input parameters for value data
server_url = "opc.tcp://10.241.80.4:4872"
# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z" # 5th July aggregated data 
end_time = "2022-07-06T07:55:14.544000Z"
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

pro_interval1 = 3600000 # 1 hour processing time
end_time1 = "2022-07-10T07:55:14.544000Z" # 5 days
end_time2 = "2022-07-15T07:55:14.544000Z" # 10 days
end_time3 = "2022-08-05T07:55:14.544000Z" # 30 days

INVERTERS

In [7]:
inverters = mdx.get_object_descendants("InverterType", ['13:1:SSO.JO-GL'], "PV_Assets")

In [8]:
# Node ids of the inverters Nodes
inv_node_ids = mdx.get_vars_node_ids(inverters) 

In [9]:
# Convert the json data into dataframe
inverters_df = mdx.expand_props_vars(inverters)

There are 12 inverters and 300 inverter variables' node ids for the site.

In [10]:
# List of unique inverter variables
inv_var_list = ['ACActivePower', 'ACCurrentAverage', 'ACCurrentPhaseA', 'ACCurrentPhaseB', 'ACCurrentPhaseC',
'ACDailyActiveEnergy', 'ACReactivePower', 'ACTotalActiveEnergy', 'ACVoltagePhaseAB', 'ACVoltagePhaseAC', 'ACVoltagePhaseBC', 'DCCurrent', 'DCPower', 'DCVoltage', 'Frequency', 'PowerFactor',
'Status', 'TemperatureCoil', 'TemperatureCoolingIn', 'TemperatureCore']

STRINGS

In [1]:
# Strings set type data 
strings = mdx.get_object_descendants("StringSetType", ['13:1:SSO.JO-GL'], "PV_Assets")

NameError: name 'mdx' is not defined

In [12]:
# Node ids of the strings Nodes
strngs_node_ids = mdx.get_vars_node_ids(strings) 

In [13]:
# Convert the json data into dataframe
strings_df = mdx.expand_props_vars(strings)

902 strings are there for the site and with 14 variables for each string, 12628 variables node ids present. 

In [14]:
strng_var_list = ['DCCurrent', 'DCPower', 'DCVoltage', 'RelativeDCPower', 'Status']

TRACKERS

In [16]:
# Object ids of strings set type objects to get trackers data
string_objects_ids = [x['DescendantId'] for x in strings]

In [17]:
# Tracker type data 
trackers = mdx.get_object_ancestors("TrackerType", string_objects_ids, "PV_Serves")

In [18]:
# Node ids of the trackers Nodes
tracker_ids = mdx.get_vars_node_ids(trackers) 

In [19]:
tracker_node_ids = pd.unique(tracker_ids)

In [20]:
# Convert the json data into dataframe
trackers_df = mdx.expand_props_vars(trackers)

There are 165 trackers are plotted on JO-GL site and each tracker has 4 variables to measure four different parameters. (But more than one string can be under one tracker)

In [21]:
track_var_list = ['TrackingLimitEastAngle', 'TrackingLimitWestAngle', 'AngleMeasured', 'AngleSetpoint']

In [22]:
track_var_list1 = track_var_list[3:5]

- Inverters' variables node ids: 300 
- Strings' variables node ids: 12628 
- Trackers' variables node ids: 650

Live Value

In [23]:
# Live value inverter data for all the variables
inv_liv_values = opc.get_live_values(server_url, inv_var_list, inv_node_ids)

Live data values of 12 inverters for all variables (300 node ids) took 0.7 seconds

In [24]:
# Live value strings data for all the variables
strng_liv_values = opc.get_live_values(server_url, strng_var_list, strngs_node_ids)

While strings live value data for 12628 node ids returned in 0.8 seconds.

In [25]:
# Live value trackers data for all the variables
track_liv_values = opc.get_live_values(server_url, track_var_list, tracker_node_ids)

Trackers live value data for 350 node ids returned in 0.3 seconds.

Inverters aggregated historical data

In [26]:
# 1 day aggregated historical inverter data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time, pro_interval, agg_name, inv_node_ids, inv_var_list, 100000,1000,10)
await session.close()

Success: 0


1.6 seconds

In [ ]:
# 10 day aggregated historical inverter data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time2, pro_interval, agg_name, inv_node_ids, inv_var_list, 100000,1000,10)
await session.close()

6.9 seconds

In [ ]:
# 1 month aggregated historical inverter data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time3, pro_interval, agg_name, inv_node_ids, inv_var_list, 100000,1000,10)
await session.close()

41.2 seconds

Strings aggregated historical data

In [ ]:
# 1 day aggregated historical strings data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time, pro_interval, agg_name, strngs_node_ids, strng_var_list, 100000,1000,10)
await session.close()

Around 1 minute 29 seconds

In [ ]:
# 10 day aggregated historical strings data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time2, pro_interval, agg_name, strngs_node_ids, strng_var_list, 100000,1000,10)
await session.close()

4 minutes 48.6 seconds

In [43]:
# 1 month aggregated historical strings data of site JO-GL for specific variables (1 hour processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time3, pro_interval, agg_name, strngs_node_ids, strng_var_list, 100000,1000,10)
await session.close()

Success: 0


1 minute 45.5 seconds.

Trackers aggregated historical data

In [45]:
# 1 day aggregated historical trackers data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time, pro_interval, agg_name, tracker_node_ids, track_var_list, 100000,1000,10)
await session.close()

Success: 0


23.1 seconds

In [46]:
# 10 day aggregated historical trackers data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time2, pro_interval, agg_name, tracker_node_ids, track_var_list, 100000,1000,10)
await session.close()

Success: 0


1 minute 8.3 seconds

In [48]:
# 1 month aggregated historical trackers data of site JO-GL for all the variables (10 minutes processing time)
session = ClientSession()
await opc.get_agg_hist_value_chunks_parallel(session,server_url, start_time, end_time3, pro_interval, agg_name, tracker_node_ids, track_var_list, 100000,1000,10)
await session.close()

Success: 0


1 minutes 29.5 seconds for two variables

Observations

Writting data into files while loading improve the efficiency. Since I do not store it in memory so, I can make multiple requests for different datasets. 
- Big requests happens in loop so there wouldn't be server connection timed out error. 